<a href="https://colab.research.google.com/github/FatmaBuseBorlu/Yapay-Zeka-DREAMER/blob/main/gemini_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# EEG İLE DUYGU TANIMA PROJESİ - NİHAİ, TAM VE ÇALIŞAN KOD (v3)
# ==============================================================================

# ------------------------------------------------------------------------------
# Gerekli Kütüphaneler
# ------------------------------------------------------------------------------
import numpy as np
import scipy.io
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
import time
import os

# Google Drive'ı bağla
from google.colab import drive
drive.mount('/content/drive')

# ------------------------------------------------------------------------------
# Yapılandırma
# ------------------------------------------------------------------------------
FILEPATH = '/content/drive/MyDrive/Colab Notebooks/EEG Tabanlı Duygu Tanımada Hibrit CNN-LSTM Modellerinin Başarımı: DREAMER Veri Seti Üzerine Bir Analiz/DREAMER.mat'
TARGET_EMOTION = 'Arousal'
EPOCHS = 20
BATCH_SIZE = 128
SEGMENT_LENGTH = 256

# ------------------------------------------------------------------------------
# 1. VERİ YÜKLEME FONKSİYONU (TÜM YAPISAL SORUNLAR ÇÖZÜLMÜŞ)
# ------------------------------------------------------------------------------
def load_and_process_dreamer_data(filepath):
    """
    Standart DREAMER.mat dosyasını, tüm karmaşık iç içe geçmiş yapıları
    doğru şekilde işleyerek yükler ve baseline düzeltmesi yapar.
    """
    if not os.path.exists(filepath):
        print(f"HATA: Dosya bulunamadı: {filepath}")
        return None, None, None

    mat = scipy.io.loadmat(filepath)
    all_eeg_data, all_labels_valence, all_labels_arousal = [], [], []

    try:
        data_struct = mat['DREAMER'][0, 0]['Data'][0]
    except KeyError:
        print("HATA: Dosya yapısı beklenenden farklı. 'DREAMER' veya 'Data' anahtarı bulunamadı.")
        return None, None, None

    for subject_id in range(len(data_struct)):
        subject = data_struct[subject_id]
        subject_eeg_list, subject_labels_valence, subject_labels_arousal = [], [], []

        for video_id in range(18):
            try:
                stimuli_signal = subject['EEG'][0,0]['stimuli'][0,0][video_id,0]
                baseline_signal = subject['EEG'][0,0]['baseline'][0,0][video_id,0]

                # --- NİHAİ DÜZELTME: Skorları en derindeki paketinden çıkarma ---
                valence_val = subject['ScoreValence'][0,0][video_id,0]
                arousal_val = subject['ScoreArousal'][0,0][video_id,0]
                # --- DÜZELTME SONU ---

            except (IndexError, KeyError) as e:
                print(f"Uyarı: Katılımcı {subject_id+1}, Video {video_id+1} için veri okunamadı: {e}. Atlanıyor.")
                continue

            if stimuli_signal.ndim == 2 and baseline_signal.ndim == 2:
                baseline_mean = np.mean(baseline_signal, axis=0)
                eeg_baselined = stimuli_signal - baseline_mean

                subject_eeg_list.append(eeg_baselined)
                subject_labels_valence.append(int(valence_val))
                subject_labels_arousal.append(int(arousal_val))

        all_eeg_data.append(subject_eeg_list)
        all_labels_valence.append(subject_labels_valence)
        all_labels_arousal.append(subject_labels_arousal)

    return all_eeg_data, all_labels_valence, all_labels_arousal

def create_segments(eeg_signals, labels, window_size):
    segments, segment_labels = [], []
    overlap = window_size // 2
    for i in range(len(eeg_signals)):
        signal, label = eeg_signals[i], labels[i]
        for start in range(0, signal.shape[0] - window_size, window_size - overlap):
            segments.append(signal[start : start + window_size, :])
            segment_labels.append(1 if label > 3 else 0)
    return np.array(segments), np.array(segment_labels)

# ------------------------------------------------------------------------------
# 2. MODEL TANIMLAMA FONKSİYONLARI
# ------------------------------------------------------------------------------
def build_cnn_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv1D(filters=32, kernel_size=3, activation='relu')(inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(filters=64, kernel_size=3, activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_lstm_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = LSTM(64, return_sequences=True)(inputs)
    x = LSTM(32)(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_cnn_lstm_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv1D(filters=32, kernel_size=3, activation='relu')(inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = LSTM(64)(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ------------------------------------------------------------------------------
# 3. ANA EĞİTİM VE DEĞERLENDİRME KODU
# ------------------------------------------------------------------------------
print("DREAMER standart veri seti yükleniyor...")
eeg_signals, valence_labels, arousal_labels = load_and_process_dreamer_data(FILEPATH)

if eeg_signals:
    print("✅ Veri yüklemesi başarıyla tamamlandı.")
    target_labels = arousal_labels if TARGET_EMOTION == 'Arousal' else valence_labels
    print(f"Hedef duygu olarak '{TARGET_EMOTION}' seçildi.")

    all_scores = { "CNN": [], "LSTM": [], "CNN-LSTM": [] }
    loo = LeaveOneOut()
    subjects = list(range(len(eeg_signals)))

    for fold, (train_indices, test_indices) in enumerate(loo.split(subjects)):
        test_subject_id = test_indices[0]
        print(f"\n===== FOLD {fold+1}/{len(subjects)} | Test Katılımcısı: {test_subject_id+1} =====")

        X_train_list, y_train_list = [], []
        for sub_idx in train_indices:
            segments, labels = create_segments(eeg_signals[sub_idx], target_labels[sub_idx], SEGMENT_LENGTH)
            if len(segments) > 0:
                X_train_list.append(segments)
                y_train_list.append(labels)

        X_test, y_test = create_segments(eeg_signals[test_subject_id], target_labels[test_subject_id], SEGMENT_LENGTH)

        if not X_train_list or len(X_test) == 0:
            print("Eğitim veya test için yeterli veri yok, bu fold atlanıyor.")
            continue

        X_train = np.concatenate(X_train_list, axis=0)
        y_train = np.concatenate(y_train_list, axis=0)

        scaler = StandardScaler()
        X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
        X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])

        X_train = scaler.fit_transform(X_train_reshaped).reshape(X_train.shape)
        X_test = scaler.transform(X_test_reshaped).reshape(X_test.shape)

        y_train_cat = to_categorical(y_train, num_classes=2)

        input_shape = (X_train.shape[1], X_train.shape[2])
        models = {
            "CNN": build_cnn_model(input_shape, 2),
            "LSTM": build_lstm_model(input_shape, 2),
            "CNN-LSTM": build_cnn_lstm_model(input_shape, 2)
        }

        for name, model in models.items():
            print(f"\n--- {name} modeli eğitiliyor... ---")
            model.fit(X_train, y_train_cat, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)
            y_pred = np.argmax(model.predict(X_test), axis=1)
            acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            all_scores[name].append({'acc': acc, 'f1': f1})
            print(f"-> {name} Modeli Sonuçları: Doğruluk={acc:.4f}, F1-Skor={f1:.4f}")

    print(f"\n\nTüm eğitim süreci tamamlandı.")
    print("\n\n===== TÜM KATILIMCILAR İÇİN ORTALAMA PERFORMANS SONUÇLARI =====")
    for name in all_scores:
        if all_scores[name]:
            avg_acc = np.mean([s['acc'] for s in all_scores[name]])
            std_acc = np.std([s['acc'] for s in all_scores[name]])
            print(f"Model: {name}\t Ortalama Doğruluk: {avg_acc:.4f} (± {std_acc:.4f})")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
DREAMER standart veri seti yükleniyor...
✅ Veri yüklemesi başarıyla tamamlandı.
Hedef duygu olarak 'Arousal' seçildi.

===== FOLD 1/23 | Test Katılımcısı: 1 =====

--- CNN modeli eğitiliyor... ---
116/116 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step
-> CNN Modeli Sonuçları: Doğruluk=0.5450, F1-Skor=0.5612

--- LSTM modeli eğitiliyor... ---
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
-> LSTM Modeli Sonuçları: Doğruluk=0.4748, F1-Skor=0.4876

--- CNN-LSTM modeli eğitiliyor... ---
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
-> CNN-LSTM Modeli Sonuçları: Doğruluk=0.5685, F1-Skor=0.5840

===== FOLD 2/23 | Test Katılımcısı: 2 =====

--- CNN modeli eğitiliyor... ---
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
-> CNN Modeli Sonuçları: Doğruluk=0.4846, F1-Skor=0.4666

--- LSTM modeli eğitiliyor... ---
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
-> LSTM Modeli Sonuçları: Doğruluk=0.4564, F1-